In [1]:
# ✅ Enhanced ResNet-50 Training Script to Avoid Overfitting (Full Pipeline)

import os
import time
import copy
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split, WeightedRandomSampler
from tqdm import tqdm

# Seed fix
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Dataset path
root_dir = '/kaggle/input/bollywood-celeb-localized-face-dataset-extended/dataset'

# Transforms
transform = {
    "train": transforms.Compose([
        transforms.RandomResizedCrop(224, scale=(0.6, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(20),
        transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
        transforms.RandomGrayscale(p=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    "val": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

# Load dataset
full_dataset = datasets.ImageFolder(root=root_dir, transform=transform["train"])
num_classes = len(full_dataset.classes)
print(f"Detected {num_classes} classes")

train_size = int(0.7 * len(full_dataset))
val_size = int(0.15 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(SEED)
)
val_dataset.dataset.transform = transform["val"]
test_dataset.dataset.transform = transform["val"]

# Class-balanced sampler
labels = [full_dataset[i][1] for i in train_dataset.indices]
class_sample_count = np.bincount(labels)
weights = 1. / class_sample_count[labels]
sampler = WeightedRandomSampler(weights, len(weights))

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# ResNet-50 with dropout
model = models.resnet50(pretrained=True)
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.fc.in_features, num_classes)
)
model = model.to(device)

# Freeze all layers initially
for param in model.parameters():
    param.requires_grad = False
for name, param in model.named_parameters():
    if "layer4" in name or "fc" in name:
        param.requires_grad = True

# Optimizer and scheduler
params_to_update = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.Adam(params_to_update, lr=5e-5)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=2, factor=0.5, verbose=True)

# Mixup function
def mixup_data(x, y, alpha=1.0):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(x.device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

# Training loop
num_epochs = 50
best_val_acc = 0
no_improve_epochs = 0
patience = 7
best_weights = copy.deepcopy(model.state_dict())

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch + 1}/{num_epochs}")
    start = time.time()

    model.train()
    train_correct = 0
    train_total = 0
    train_loop = tqdm(train_loader, desc="Training", leave=False)

    for images, labels in train_loop:
        images, labels = images.to(device), labels.to(device)
        images, targets_a, targets_b, lam = mixup_data(images, labels)
        outputs = model(images)
        loss = mixup_criterion(criterion, outputs, targets_a, targets_b, lam)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _, preds = torch.max(outputs, 1)
        train_total += labels.size(0)
        train_correct += (lam * preds.eq(targets_a).sum().item() + (1 - lam) * preds.eq(targets_b).sum().item())
        train_loop.set_postfix(loss=loss.item())

    train_acc = train_correct / train_total

    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        val_loop = tqdm(val_loader, desc="Validating", leave=False)
        for images, labels in val_loop:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)
    val_acc = val_correct / val_total

    print(f"Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Time: {time.time()-start:.1f}s")

    scheduler.step(val_acc)

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_weights = copy.deepcopy(model.state_dict())
        no_improve_epochs = 0
    else:
        no_improve_epochs += 1
        if no_improve_epochs >= patience:
            print("Early stopping triggered.")
            break

# Load best model
model.load_state_dict(best_weights)

# Evaluate on test set
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    test_loop = tqdm(test_loader, desc="Testing", leave=False)
    for images, labels in test_loop:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        test_correct += (preds == labels).sum().item()
        test_total += labels.size(0)
print(f"\n✅ Final Test Accuracy: {test_correct / test_total:.4f}")

Using device: cuda
Detected 170 classes


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 191MB/s] 
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn


Epoch 1/50


Train Acc: 0.0255, Val Acc: 0.0598, Time: 45.9s

Epoch 2/50


Train Acc: 0.0676, Val Acc: 0.1431, Time: 47.1s

Epoch 3/50


...

Train Acc: 0.7280, Val Acc: 0.7492, Time: 50.1s

Epoch 48/50


Train Acc: 0.7206, Val Acc: 0.7497, Time: 50.1s

Epoch 49/50


Train Acc: 0.7255, Val Acc: 0.7410, Time: 50.1s

Epoch 50/50


Train Acc: 0.7267, Val Acc: 0.7486, Time: 50.0s



✅ Final Test Accuracy: 0.7679
